In [1]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence



import sys
if ".." not in sys.path:
    sys.path.insert(0, "..")

from datasets import create_batchdict, PlayByPlayDataset

dataset = PlayByPlayDataset("../data/nfl-big-data-bowl-2024/play_by_play_val.pkl")


torch.__version__


'2.1.0'

In [8]:
import numpy as np

y = np.array([dataset[i]['yards_after_contact'] for  i  in range(len(dataset))])
np.mean(y)


3.4594572025052197

In [8]:
dataset[0]["play_features_encoded"]


Q1                       0.000000
Q2                       0.000000
Q3                       0.000000
Q4                       1.000000
OT                       0.000000
down_1                   0.000000
down_2                   0.000000
down_3                   1.000000
down_4                   0.000000
yardage_short            0.000000
yardage_medium           1.000000
yardage_long             0.000000
yardage_very_long        0.000000
time                     3.066667
preSnapHomeScore        19.000000
preSnapVisitorScore      3.000000
0                       16.000000
formation_EMPTY          0.000000
formation_I_FORM         0.000000
formation_JUMBO          0.000000
formation_PISTOL         0.000000
formation_SHOTGUN        0.000000
formation_SINGLEBACK     1.000000
formation_WILDCAT        0.000000
n_def_in_box_(0, 4]      0.000000
n_def_in_box_(4, 5]      0.000000
n_def_in_box_(5, 6]      0.000000
n_def_in_box_(6, 7]      1.000000
n_def_in_box_(7, 8]      0.000000
n_def_in_box_(

In [4]:
from datasets import *

def collate_padded_play_data_with_context(batch):
    batchdict = create_batchdict(batch)
    X_geometric = torch.cat([pad_sequence(batchdict[k], batch_first=True, padding_value=PAD_VALUE) for k in GEOMETRIC_KEYS + RAW_KEYS], dim=2)
    X_ball_carrier = pad_sequence(batchdict["ball_carrier_raw"], batch_first=True, padding_value=PAD_VALUE)
    X_tacklers, n_tacklers = pad_tacklers(batchdict["tacklers_raw"])
    import pdb; pdb.set_trace()
    X_padded_static = None # TODO: tile in time, then pad_sequence
    return {
        "time_series_features": (X_geometric, X_ball_carrier, X_tacklers, n_tacklers),
        "features": X_padded_static,
        "target": torch.tensor(batchdict[TARGET_KEY], dtype=torch.float),
        "treatment": torch.tensor(batchdict[TREATMENT_KEY], dtype=torch.float),
    }

def collate_padded_play_data_with_carrier_tackler_and_raw_info(batch):
    batchdict = create_batchdict(batch)
    X_geometric = torch.cat([pad_sequence(batchdict[k], batch_first=True, padding_value=PAD_VALUE) for k in GEOMETRIC_KEYS + RAW_KEYS], dim=2)
    X_ball_carrier = pad_sequence(batchdict["ball_carrier_raw"], batch_first=True, padding_value=PAD_VALUE)
    X_tacklers, n_tacklers = pad_tacklers(batchdict["tacklers_raw"])
    return {
        "time_series_features": (X_geometric, X_ball_carrier, X_tacklers, n_tacklers),
        "target": torch.tensor(batchdict[TARGET_KEY], dtype=torch.float),
        "treatment": torch.tensor(batchdict[TREATMENT_KEY], dtype=torch.float),
    }

dataloader = DataLoader(dataset, batch_size=4, shuffle=False, collate_fn=collate_padded_play_data_with_carrier_tackler_and_raw_info)
batch = next(iter(dataloader))


tensor([ 1.9286e+01,  1.0480e+01,  3.8338e+00,  8.9530e+01,  5.3906e-03,
         3.4318e+01,  8.2177e+00,  2.5561e+00,  2.9008e+02,  4.9455e-01,
         3.0408e+01,  8.6422e+00,  4.0009e+00,  2.9867e+02, -9.7618e-01,
         3.2574e+01,  9.1777e+00,  3.7400e+00,  2.2916e+02, -9.8451e-01,
         2.3795e+01,  7.2280e+00,  2.1739e+00, -5.4800e+00,  6.9442e-01,
         1.3922e+01,  7.0265e+00,  2.5799e+00, -3.0410e+01,  5.3531e-01,
         2.3409e+01,  7.9154e+00,  2.7080e+00, -2.4020e+01,  4.4220e-01,
         3.3115e+01,  8.0008e+00,  2.8082e+00,  1.3324e+02,  2.7413e-01,
         1.4908e+01,  6.9230e+00,  2.5843e+00, -7.2000e+00,  6.0835e-01,
         3.0462e+01,  8.9044e+00,  4.1169e+00, -2.7630e+01, -7.9950e-01,
         3.5449e+01,  7.3952e+00,  2.3615e+00,  1.8940e+02,  6.1813e-01,
         1.4619e+01,  1.3415e+01,  4.6246e+00,  7.7720e+01, -6.8236e-01,
         9.7135e+00,  1.6072e+01,  7.2316e+00, -9.7600e+00, -9.4434e-01,
         1.6783e+01,  9.1103e+00,  3.6585e+00, -1.1

In [25]:
first_X = batch["time_series_features"][0][0, 0, :]
n_feats = len(first_X)
raw_feature_start_idx = (N_OFFENSE + N_DEFENSE) * N_GEOMETRIC_FEATS
new_indices = (torch.arange(n_feats) < raw_feature_start_idx) | (torch.remainder(torch.arange(n_feats) - raw_feature_start_idx, N_FEATS_PER_PLAYER_PER_TIMESTEP) != 0)
final_first_X = first_X[new_indices]


first_off_raw = dataset[0]["offense_raw"].iloc[0]
first_def_raw = dataset[0]["defense_raw"].iloc[0]
first_X_raw = torch.cat([
    torch.from_numpy(dataset[0]["offense_geometric"].iloc[0].values),
    torch.from_numpy(dataset[0]["defense_geometric"].iloc[0].values),
    torch.from_numpy(first_off_raw[first_off_raw.index.get_level_values(1) != "x"].values),
    torch.from_numpy(first_def_raw[first_def_raw.index.get_level_values(1) != "x"].values),
    ])

(final_first_X == first_X_raw).all()


tensor(True)

In [22]:
first_off_raw = dataset[0]["offense_raw"].iloc[0]
first_off_raw[first_off_raw.index.get_level_values(1) != "x"]


42363.0  y       26.37
         s        6.43
         a        2.50
         o      270.72
         dir    229.18
42587.0  y       23.64
         s        0.19
         a        1.14
         o      302.45
         dir     28.63
42883.0  y       26.28
         s        0.34
         a        1.10
         o      251.99
         dir     20.04
44912.0  y       28.53
         s        0.88
         a        0.83
         o      313.68
         dir     89.55
46092.0  y       32.34
         s        1.71
         a        1.47
         o      271.12
         dir    324.19
48541.0  y       38.00
         s        4.77
         a        2.32
         o      333.39
         dir    349.12
52590.0  y       31.45
         s        1.01
         a        1.98
         o      353.47
         dir    342.73
53444.0  y       24.22
         s        2.66
         a        0.81
         o      353.43
         dir    185.47
53549.0  y       41.83
         s        7.16
         a        2.92
         o 

In [4]:
test_batch_size = 4
sample_batchdict = create_batchdict([dataset[i] for i in range(test_batch_size)])

[t.view(t.size(0), -1, 6).size() for t in sample_batchdict["tacklers_raw"]]


[torch.Size([43, 1, 6]),
 torch.Size([21, 2, 6]),
 torch.Size([16, 1, 6]),
 torch.Size([47, 1, 6])]

In [16]:
(batch['time_series_features'][0].size(),
batch['time_series_features'][1].size(),
batch['time_series_features'][2].size(),
batch['time_series_features'][3],
)


(torch.Size([4, 47, 105]),
 torch.Size([4, 47, 6]),
 torch.Size([4, 47, 2, 6]),
 tensor([1, 2, 1, 1]))

In [59]:
# 248 is the dimensionality of the time-series features

encoder_layer = nn.TransformerEncoderLayer(d_model=248, nhead=8, batch_first=True)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
X = batch["time_series_features"].float()
out = transformer_encoder(X, src_key_padding_mask=(X[..., 0] == PAD_VALUE))


In [60]:
out.size() # batch size, time, n_features
# in the future -- concat with batch_size, n_features' shaped things -- need to squish out the time dim


torch.Size([8, 39, 248])

In [61]:
39 * 248


9672